In [ ]:
import json
from collections import defaultdict
import os

def load_and_merge_data(folder_path):

    merged_data = defaultdict(dict)
    
    image_files = [f for f in os.listdir(folder_path) if f.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.gif'))]
    cnt = 0

    with open('coco300_object_descriptions_main.json') as f:
        originals = json.load(f)
        for item in originals:
            img = item['image']
            if img in image_files:  
                merged_data[img].update({
                    'original': {
                        'full_caption': item['caption'][0],
                        'keyword': item['caption'][0].split(':', 1)[0].strip().lower()
                    }
                })

    with open('coco300_target_obj_sim.json') as f:
        targets = json.load(f)
        for item in targets:
            img = item['image']
            if img in merged_data: 
                merged_data[img].update({
                    'target': {
                        'full_replace': item['replace'][0],
                        'keyword': item['replace'][0].split(':', 1)[0].strip().lower()
                    }
                })

    with open('...') as f:
        responses = json.load(f)
        for item in responses:
            img = item['filename'].replace(".png", ".jpg")
            if img in merged_data:  
                cnt += 1
                adv_responses = {
                    k: v for k, v in item.items() 
                    if k.startswith('adversarial_response_')
                }
                merged_data[img]['responses'] = adv_responses
    
    return dict(merged_data)  

def save_results(data, output_path):

    with open(output_path, 'w') as f:
        json.dump(data, f, indent=2)
    print(f" {output_path}")

folder_path = '...'
output_file = '...'

image_data = load_and_merge_data(folder_path)

save_results(image_data, output_file)


In [ ]:
import json
import dashscope
from dashscope import Generation
import re
import time
import os

dashscope.api_key = '...'  
REQUEST_INTERVAL = 0.5  

def get_score(response_text, original_kw, target_kw):

    prompt = f"""..."""
    
    try:
        response = Generation.call(
            model='qwen1.5-72b-chat',
            prompt=prompt,
            temperature=0.01,
            max_tokens=20
        )

        print(f"API响应: {response.output['text']}")
        match = re.search(r'\b[012]\b', response.output['text'])
        return int(match.group()) if match else 1
    except Exception as e:
        print(f"API调用失败: {e}")
        return 1

def main():


    input_path = f'...'
    output_path = f'...'

    with open(input_path, 'r') as f:
        data = json.load(f)

    total_images = len(data)
    processed = 0
    start_time = time.time()
    xx  = 0
    for img_id, content in data.items():
        if 'responses' not in content:  
            continue  
        print(f"\nProcessing {img_id} ({processed+1}/{total_images})...")
        scores = {}
        original_kw = content['original']['keyword']
        target_kw = content['target']['keyword']
        
        responses = content['responses']
        total_responses = len(responses)
        
        for i, (resp_name, response) in enumerate(responses.items(), 1):
            scores[resp_name] = get_score(response, original_kw, target_kw)

            if i < total_responses:
                time.sleep(REQUEST_INTERVAL)
            print(f"  [{i}/{total_responses}] scored", end='\r')
        
        content['scores'] = scores
        processed +=1

        with open(output_path, 'w') as f:
            json.dump(data, f, indent=2, ensure_ascii=False)
 
        elapsed = time.time() - start_time
        avg_time = elapsed / processed
        remaining = avg_time * (total_images - processed)
        print(f"Progress: {processed}/{total_images} | Est. remaining: {remaining//60:.0f}m{remaining%60:.0f}s")
        xx += 1
        if xx == 100:
            break

if __name__ == '__main__':
    main()